# Build the Neural Network

[Link to the related offical tutorial](https://docs.pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html). Based by Update:: Jan 24, 2025

Neural networks comprise of layers/modules that perform operations on data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we’ll build a neural network to classify images in the FashionMNIST dataset.

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Get Device for Training

We want to be able to train our model on an [accelerator](https://pytorch.org/docs/stable/torch.html#accelerators) such as CUDA, MPS, MTIA, or XPU. If the current accelerator is available, we will use it. Otherwise, we use the CPU.

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


# Define the Class

We define our neural network by subclassing `nn.Module`, and initialize the neural network layers in `__init__`. Every `nn.Module` subclass implements the operations on input data in the `forward` method.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`, and print its structure.

In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model’s `forward`, along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866). Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the `nn.Softmax` module.

In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7])


# Model Layers

Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


# nn.Flatten

We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


# nn.Linear

The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) is a module that applies a linear transformation on the input using its stored weights and biases.

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


# nn.ReLU

Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our linear layers, but there’s other activations to introduce non-linearity in your model.

In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1246, -0.5447, -0.2723, -0.2683,  0.0014, -0.1697, -0.3985,  0.5260,
          0.1221, -0.2756,  0.4873, -0.1769, -0.3930,  0.1267,  0.1994,  0.6850,
         -0.2002,  0.2494, -0.1476,  0.2757],
        [ 0.0096, -0.7805, -0.5840, -0.1280, -0.1715, -0.3006, -0.3051,  0.1480,
          0.2628, -0.7782,  0.0076, -0.0133, -0.5557,  0.2303,  0.5479,  0.0602,
          0.0637,  0.5342, -0.3298,  0.3535],
        [ 0.2356, -0.6195, -0.0212, -0.3030,  0.2449, -0.2861, -0.1756,  0.3741,
          0.3835, -0.3160,  0.2109,  0.0772, -0.4899,  0.1329,  0.2629,  0.0352,
         -0.1810,  0.2937, -0.1387,  0.4474]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0014, 0.0000, 0.0000, 0.5260, 0.1221,
         0.0000, 0.4873, 0.0000, 0.0000, 0.1267, 0.1994, 0.6850, 0.0000, 0.2494,
         0.0000, 0.2757],
        [0.0096, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1480, 0.2628,
         0.0000, 0.0076, 0.0000, 0.0000, 0.2303, 0.54

# nn.Sequential

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like `seq_modules`.

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

input_image, logits

(tensor([[[0.7678, 0.3577, 0.7353,  ..., 0.0423, 0.5738, 0.4122],
          [0.4902, 0.3629, 0.5518,  ..., 0.2843, 0.7921, 0.9271],
          [0.3213, 0.4581, 0.1752,  ..., 0.1405, 0.9648, 0.6494],
          ...,
          [0.7226, 0.0259, 0.0072,  ..., 0.9213, 0.1015, 0.0699],
          [0.9119, 0.2159, 0.2805,  ..., 0.3655, 0.0987, 0.5805],
          [0.4949, 0.6785, 0.9528,  ..., 0.8427, 0.0153, 0.5446]],
 
         [[0.0855, 0.7375, 0.9145,  ..., 0.5742, 0.3839, 0.1516],
          [0.9692, 0.9947, 0.6562,  ..., 0.6730, 0.9904, 0.7877],
          [0.2156, 0.1273, 0.4750,  ..., 0.6029, 0.1687, 0.3495],
          ...,
          [0.1844, 0.0099, 0.9179,  ..., 0.5508, 0.7832, 0.2717],
          [0.8393, 0.3042, 0.7636,  ..., 0.2134, 0.8421, 0.4424],
          [0.8583, 0.6771, 0.9145,  ..., 0.0959, 0.6862, 0.5294]],
 
         [[0.1219, 0.4887, 0.4164,  ..., 0.4732, 0.5373, 0.4662],
          [0.8333, 0.3444, 0.0970,  ..., 0.8987, 0.1803, 0.5006],
          [0.8669, 0.2837, 0.6210,  ...,

# nn.Softmax

The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

pred_probab

tensor([[0.0903, 0.0957, 0.0973, 0.1054, 0.1078, 0.0836, 0.1115, 0.1249, 0.0969,
         0.0866],
        [0.0868, 0.0915, 0.1003, 0.1015, 0.1070, 0.0799, 0.1091, 0.1326, 0.1057,
         0.0856],
        [0.0903, 0.0952, 0.0993, 0.1018, 0.1046, 0.0806, 0.1164, 0.1215, 0.0976,
         0.0926]], grad_fn=<SoftmaxBackward0>)

# Model Parameters

Many layers inside a neural network are *parameterized*, i.e. have associated weights and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0062, -0.0178, -0.0327,  ...,  0.0063, -0.0132,  0.0128],
        [-0.0128,  0.0200, -0.0029,  ...,  0.0302, -0.0164, -0.0269]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0057, -0.0257], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0190,  0.0243, -0.0012,  ..., -0.0224, -0.0251,  0.0277],
        [ 0.0355,  0.0387,  0.0048,  ..., -0.0108, -0.0022,  0.0171]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 

# Further Reading

- [torch.nn API](https://pytorch.org/docs/stable/nn.html)

